In [1]:
# import libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from utils.gengraph import gen_syn1, gen_syn3, gen_syn4, preprocess_input_graph

import networkx as nx
import matplotlib.pyplot as plt

# Define GCN Classes

In [2]:
class GraphConvolutionLayer(nn.Module):
    """
    A single Graph Convolution Layer
    """
    def __init__(self, dim_in:int, dim_out:int):
        super().__init__()
        
        self.dim_in = dim_in # input number of features
        self.dim_out = dim_out # output size
        # Initialize weights as suggested, using Glorot initialization
        self.W = nn.Parameter(
            nn.init.xavier_uniform_(torch.empty(self.dim_in,
                                                self.dim_out,
                                                dtype=torch.float64)))
    
    def forward(self, H:torch.sparse, A:torch.sparse):# -> torch.Tensor:
        
        # compute degree matrix:
        # 1. Combine sum over rows and sum over columns
        # 2. Remove identity vector from sums to deal with double self-loops
        # 3. Take inverse square root of all elements and create diagonal matrix
        self.D = torch.diag(1/torch.sqrt(
            (-torch.ones(A.shape[0]) + A.sum(axis=0) + A.sum(axis=1))/2))
        
        # Compute Â
        self.A_hat = self.D @ A @ self.D
        
        return (self.A_hat @ H @ self.W) 


class GCN(nn.Module):
    def __init__(self, dim_in, dim_hidden, dim_out):
        super().__init__()

        self.dim_in = dim_in
        self.dim_hidden = dim_hidden
        self.dim_out = dim_out
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        
        self.GCN1 = GraphConvolutionLayer(dim_in, dim_hidden)
        self.GCN2 = GraphConvolutionLayer(dim_hidden, dim_out)

    def encode(self, H:torch.sparse, A:torch.sparse) -> torch.Tensor:
        h1 = self.relu(self.GCN1(H, A))
        h1 = self.dropout(h1)
        return self.GCN2(h1, A)
    
    def decode(self, node_embeds, edge_ids):
        """ Compute edge embeddings based on node similarity
        
        Edge_ids stores all pairs of node ids for which we want link predictions
        
        ::param:: node_embeds <- torch.tensor(n_nodes, size_embeds)
        ::param:: edge_ids <- torch.tensor(n_edges, 2)
        
        Return:
            edge_embeds <- torch.tensor(n_edges, 1)
        """
        edge_embeds = torch.sum(node_embeds[edge_ids.T[0]] * node_embeds[edge_ids.T[1]], dim=-1)
        return edge_embeds

# Create synthetic dataset

In [136]:
a, b, c = gen_syn1(width_basis=400) # BA-base graph with house motifs
# a, b, c = gen_syn3() # BA-base graph with grid motif
# a, b, c = gen_syn4() # Tree-base graph with cycle motif

data = preprocess_input_graph(a, b)


feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32


In [27]:
def gen_data_split(adj, p:float=0.1):
    """ 
    Returns train sub-graph and edge indices and labels for train and test links

    ::param:: adj <- np.array : adjacency matrix
    ::param:: p <- float : proportion of positive links to remove
                           and of pos/neg links to train on
    
    Returns Tuple(torch.sparse_coo, np.array, np.array,
                  np.array, np.array):
        ...
    """
    # Find all pos edge indices in adj
    pos_edges = np.argwhere(adj == np.argmax(adj)).tolist()
    np.random.shuffle(pos_edges)
    
    # pos edges to remove
    len_split = int(len(pos_edges)*p)
    pos_ids = pos_edges[:len_split]
    pos_test = pos_edges[len_split:]
    len_test = len(pos_test)
    
    # select neg edges
    neg_edges = np.argwhere(adj == np.argmin(adj)).tolist()
    np.random.shuffle(neg_edges)
    neg_ids = neg_edges[:len_split] # same number as pos edges
    neg_test = neg_edges[len_split:len_split + len_test] # same number as pos edges
    
    sub_g = adj.copy()
    for id in pos_ids:
        sub_g[id] = 0.0
    
    return sub_g, np.concatenate((pos_ids, neg_ids)),\
        np.concatenate((np.ones(len_split), np.zeros(len_split))),\
        np.concatenate((pos_test, neg_test)),\
        np.concatenate((np.ones(len_test), np.zeros(len_test)))

In [137]:
sub_g, train_edgeids, train_labels, test_edgeids, test_labels = gen_data_split(data['adj'][0], p = 0.3)

# Train GCN on link prediction task

In [138]:
# Load full graph adjacency matrix as sparse tensor
A = torch.tensor(data['adj'][0])
A = (A + torch.eye(A.shape[0])).to_sparse_coo()

# Load sub-graph as sparse tensor
sub_g = torch.tensor(sub_g)
sub_g = (sub_g + torch.eye(sub_g.shape[0])).to_sparse_coo()

node_features = torch.eye(data['feat'][0].shape[0], dtype=torch.float64)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Define the model
input_size = node_features.size(1)
hidden_size = 500
output_size = 100  # Size of node embeddings to decode 
lr = 0.001
model = GCN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    optimizer.zero_grad()

    # Forward pass
    outputs = model.encode(node_features, A=sub_g)
    preds = model.decode(outputs, train_edgeids)
    

    # Compute the loss
    loss = criterion(preds, train_labels)
    print(f"Loss: {loss}")
    acc = torch.sum((torch.sigmoid(preds) > 0.5) == train_labels)/train_labels.shape[0]
    print(f"Accuracy: {acc}")

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

# Eval the model on test edges
print(f"----- Evaluate model -----")
model.eval()

test_outputs = model.encode(node_features, A=A)
test_preds = model.decode(test_outputs, test_edgeids)
test_loss = criterion(test_preds, test_labels)
print(f"Test Loss: {test_loss}")
test_acc = torch.sum((torch.sigmoid(test_preds) > 0.5) == test_labels)/test_labels.shape[0]
print(f"Test Accuracy: {test_acc}")


Loss: 0.7414284775989302
Accuracy: 0.4996744692325592
Loss: 0.6970053935885175
Accuracy: 0.5087890625
Loss: 0.6777322860621638
Accuracy: 0.5462239384651184
Loss: 0.6644049752254281
Accuracy: 0.6158854365348816
Loss: 0.6563650178823641
Accuracy: 0.6285807490348816
Loss: 0.6456386416664407
Accuracy: 0.6617838740348816
Loss: 0.6343771547199324
Accuracy: 0.6598307490348816
Loss: 0.6245903826080803
Accuracy: 0.673828125
Loss: 0.6104743737128465
Accuracy: 0.6809895634651184
Loss: 0.5979431568412664
Accuracy: 0.69140625
Loss: 0.5827991449259714
Accuracy: 0.7327473759651184
Loss: 0.57218910068958
Accuracy: 0.7431640625
Loss: 0.5575920822186576
Accuracy: 0.791015625
Loss: 0.542615892772539
Accuracy: 0.8212890625
Loss: 0.5276067533752035
Accuracy: 0.8421223759651184
Loss: 0.5151331950118647
Accuracy: 0.8551432490348816
Loss: 0.5014314647722502
Accuracy: 0.8720703125
Loss: 0.49080926391899976
Accuracy: 0.8834635615348816
Loss: 0.47346601568984886
Accuracy: 0.890625
Loss: 0.45876099021866173
Accur

# Define GNNExplainer

In [88]:
class GNNExplainer(nn.Module):
    """
    """
    def __init__(self, model, adj_matrix, labels, size_weight:int=0.000005):
        super(GNNExplainer, self).__init__()
        self.model = model
        self.adj_matrix = adj_matrix
        self.labels = labels
        self.size_weight = size_weight
        self.node_mask = torch.empty((adj_matrix.shape[0], adj_matrix.shape[1]), dtype=torch.float64)
        torch.nn.init.xavier_uniform_(self.node_mask)
        # torch.nn.init.constant_(self.node_mask, 1.0)
        self.node_mask = torch.sigmoid(self.node_mask)
        self.node_mask = nn.Parameter(self.node_mask)
        
    def forward(self, edge_idx, x):
        """ 
        Use node mask to mask out unimportant nodes/edges for link prediction(s).
        Returns probability for edge(s) in node_idx and final adjacency matrix mask
        
        ::param:: edge_idx <- Tuple(int, int) : indices of nodes to predict edge between
                        or <- List(Tuple(int, int)) : for multi edge prediction
        ::param:: x <- Tensor(N x k) : node features of dimension (number of nodes x number of features)
        """
        # Use trained node_mask to mask adjacency matrix
        sym_mask = self.node_mask
        sym_mask = (sym_mask + sym_mask.t())/2
        masked_adj = self.adj_matrix * sym_mask
        
        # Compute GCN prediction using masked adjacency matrix
        outputs = model.encode(x, masked_adj)
        preds = model.decode(outputs, edge_idx)
        return preds, masked_adj
    
    def loss(self, pred, explain_pred):
        entropy_loss = -torch.sum(pred * torch.log(explain_pred))
        mask_size_loss = self.size_weight * torch.sum(torch.sigmoid(self.node_mask))
            
        return entropy_loss + mask_size_loss

# Run Explainer on trained GCN

In [139]:
# Link prediction to explain
edge = torch.tensor([696, 697]) # House motif
# edge = torch.tensor([1017, 1018]) # Grid motif
# edge = torch.tensor([870, 869]) # Cycle motif

In [140]:
# Define the model
input_size = node_features.size(1)
hidden_size = 100
output_size = 1  # Output size matches the number of nodes for link prediction
lr = 0.001
explainer = GNNExplainer(model=model, adj_matrix=A, labels=...)

# Make sure GCN is set to eval
model.eval()
model.GCN1.W.requires_grad = False
model.GCN2.W.requires_grad = False

# GCN prediction
gcn_pred = torch.sigmoid(model.decode(model.encode(node_features, A), edge))

# Define loss function and optimizer
explain_optimizer = torch.optim.Adam(explainer.parameters(), lr=lr)

# Training loop (replace with your actual training loop)
num_epochs = 100
for epoch in range(num_epochs):
    explain_optimizer.zero_grad()
    
    # Forward pass
    outputs, _ = explainer(edge, node_features)

    # Compute the loss
    explain_loss = explainer.loss(outputs, gcn_pred)
    print(f"Loss: {explain_loss}")

    # Backward pass and optimization
    explain_loss.backward()
    explain_optimizer.step()


Loss: 2.2410736936887408
Loss: 2.23945775949437
Loss: 2.2378493098274372
Loss: 2.2362483566277436
Loss: 2.2346547231242813
Loss: 2.233068429937126
Loss: 2.2314894947341086
Loss: 2.22991791932833
Loss: 2.2283537171853243
Loss: 2.2267968988854845
Loss: 2.2252474636501187
Loss: 2.223705405653925
Loss: 2.2221707140576323
Loss: 2.2206433730673876
Loss: 2.2191233620183253
Loss: 2.2176106554802257
Loss: 2.2161052233833582
Loss: 2.2146070311626014
Loss: 2.213116061258851
Loss: 2.211632338227392
Loss: 2.2101557972338703
Loss: 2.208686321115435
Loss: 2.20722385386383
Loss: 2.2057680383217733
Loss: 2.2043190264668704
Loss: 2.2028770081816837
Loss: 2.201441697186585
Loss: 2.200013065241657
Loss: 2.1985910416740273
Loss: 2.1971755537136697
Loss: 2.19576652671598
Loss: 2.1943638843775117
Loss: 2.1929675489443134
Loss: 2.1915774414123796
Loss: 2.190193446513727
Loss: 2.1888153895986147
Loss: 2.1874434545342907
Loss: 2.1860774253436905
Loss: 2.1847172201139893
Loss: 2.1833627566171705
Loss: 2.18201395

# Visualize most important sub-graph

In [141]:
def filter_adj(adj, threshold=0.5):
    """ Filters graph to only include edges above threshold
    """
    filt_adj = adj.to_dense().detach().clone()
    filt_adj[adj.to_dense()<threshold] = 0
    filt_adj = filt_adj.fill_diagonal_(0) # set self-loops to 0
    return filt_adj

f = filter_adj(_)

In [143]:
fig, ax= plt.subplots(1, 1, figsize=(10,10))

# Plot subgraph
ax.set_title("")
G = nx.from_numpy_array(np.array(f))
G.remove_nodes_from(list(nx.isolates(G)))

# color predicted edge
u, v = edge
u, v = u.item(), v.item()
edge_colors = ['red' if e == (u, v) else 'black' for e in G.edges ]

# draw sub-graph
nx.draw_networkx(G, ax=ax, edge_color=edge_colors)

fig.tight_layout()
fig.savefig(f"{c}_sub_graph.png")
plt.close()